In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
import utils
from pandas import Timedelta
import time
from utils import logger, sql_to_df, run_sql, session, conn
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [ ]:
import snowflake.connector
import threading
import time
import os
class WarehousePoller:
    def __init__(self):
        # self.account = account
        # self.user = user
        # self.password = password
        self.connection = None
        self.polling_thread = None
        print("Number of active threads:", threading.active_count())

    def connect(self):
        self.connection = snowflake.connector.connect(
            user=os.environ['SNOWFLAKE_USER'],
            role=os.environ['SNOWFLAKE_ROLE'],
            password=os.environ['SNOWFLAKE_PASSWORD'],
            account=os.environ['SNOWFLAKE_ACCOUNT'],
            warehouse=os.environ['SNOWFLAKE_WAREHOUSE'],
            database=os.environ['SNOWFLAKE_DATABASE'],
            schema=os.environ['SNOWFLAKE_SCHEMA'],
            client_session_keep_alive=True
        )


    def disconnect(self):
        if self.connection is not None:
            self.connection.close()
            self.connection = None

    def poll_warehouses(self):
        while True:
            if self.connection is not None:
                cursor = self.connection.cursor()
                cursor.execute("show warehouses")
                rows = cursor.fetchall()
                print(f"Available warehouses: {rows}")
                cursor.close()
            time.sleep(10)

    def start_polling(self):
        self.polling_thread = threading.Thread(target=self.poll_warehouses)
        self.polling_thread.start()

    def stop_polling(self):
        if self.polling_thread is not None:
            self.polling_thread.join()
            self.polling_thread = None

In [ ]:
poller = WarehousePoller()
poller.connect()

In [ ]:
poller.start_polling()


In [ ]:
print("Number of active threads:", threading.active_count())

In [ ]:

# Wait a bit to see the output
# time.sleep(60)

poller.stop_polling()
poller.disconnect()